# Q Learning

or *using Graph Reinforcement Learning to automate geometric design*

Key developments:
* I developed "new" graph deep learning package (borrowed heavily from Pytorch Geometric and Google Deepmind's GraphNets paper). Like GraphNets, implements Graph Convolutional Networks, Message Passing, Deep Sets, etc. but in a very flexible API built around pytorch.
* Implement Bayesian Deep Nets and apply them to graph data.
* Demonstate ability simulate chemical reaction networks from steady state data (easy to collect). Calculates which parts we are *certain* of their behavior and which parts of the network we are *certain* of.
* Developed RL (Double-Q Learning + Bayesian + Geometric) based approach to *design* genetic circuits given a specification and learned part behavior.
* Discussion on applications of this approach for uncertain geometric design 

## Geometric and Graph Deep Learning?

Lots of things can be represented as graphs. Knowledge. Protein interactions. Social networks. Basically, a 'graph' here is a tensor of node features, a tensor of edge features, a tensor of directed edges indicating node-to-node interactions, and finally a tensor of global graph attributes.

As an example, a 3D object could be represented as a graph. Each vertex in the polygons that assemble the object may have a 3-dimensional tensor indicating its position in space. Perhaps it also has a 1-dimensional tensor indicating the distance between nodes. Maybe another global tensor indicating the category this object is (e.g. face, cat, dog, etc.). The tensor'd representation is then a 4D tensor (3d for each node), a 2D tensor (1D for each edge), and a 1D tensor (1 graph, 1 feature).

Geometric loosely here, means "represented as a graph." Typically, a graph with node and positions space and/or edges with distances or other features. I loosely use this term for all types of graph data even though it is  not accurate to distinguish the terminology from other types of graphs (such as the underlying computation graph deep learning frameworks like Pytorch use). You can think of all graphs as being 'geometric' is different types of dimensional space since we can arbitrary use different dimensions and still maintain the interactions b

**Difference between Euclidean and Geometric DL**

Traditionally, data is represented as a multi-dimensional table. 

For images, the *locality* of the data is clearly important. Hence, all of the variants of convolutional neural networks. Really, you can think of CNN as extracting local information at each position.

Going a step further, imagine you could blah blah blah.

**What does it mean to train a DL model on graphs**

Generally, we input a graphs with features and are trying to train the network to learn target graph features. This could be limited to just node features, edge_features, global features, or any combination of those. The key development here is how we relate different layers in to the interactions of the graphs. This is what my library does. Including optimized code to run quickly on a GPU (this code run painfully VERY slowly on CPUs)

**Generic Message Passaging Deep Learning**

A message passing network is a network that passes information between layers according to list of edges.

We pass edge attributes, 'aggregate' information to the node attribute later. Then aggregate the edge and node information to the global layer. We can repeat this process for as many steps as we want. We can also choose what types of layers exist at the node, edge, and global levels. We can also choose to aggregate, or not aggregate the information at each step. During the learning process, the layers will decide which information is relevant to pass from edge->node, node->global, edge->global.

[picture of edge aggregation](pic)

For example, if we choose NOT to aggregate information, we get a graph encoding of the input graphs. If we do aggregate the edge levels, we begin passing information from the edge to the node level. If we repeat the process, message are passed throughout the graph. 

**Genetic Networks as directed graphs**

This makes sense for representing something like a interaction networks in biology, which we often represent as directed network. 

[picture of interaction network]

Imagine node features could be something like GO terms, sequences. Edge features should be *how* something interacts. Global features could be organism behavior (e.g. flourescence). example 2. You can see how this type of representation is very flexible and makes a Geometric Deep Learning framework potentially very powerful for all kinds of biological data.

**In Silico simulation of learning part behavior from steady state data**

We can learn any type of generic function just from the data.

## Bayesian Deep Learning

Pyro, Uber's probalistic programming language.

I do not have time to go over the details, however, ins

Review of epistemic (model) vs aleatoric (data) uncertainty

What about "these" data points. The problem with predictive models in Deep Learning is that there is no concept of uncertainty. What this model is saying is that, "this is the most statistically likely value with unknown certainty." Are we really confortable use such a model? As far as we know, we are going to take the outputs as truth.

An alternative would be to create a deep model that not only gives use our most likely output, but also uncertainties associated with it, so we can appropriately evaluate whether we want to use the output data points.

## Reinforcement Deep Learning

## Challenges

Black-box models. A common criticism for models. However, with things like ODE models, how much does it really *explain*. Its a way to fit the data into a mental and mathematical model. However, just because the data fits the mental model, doesn't mean its an explaination. (How many times have we created a 'model' only to find out it doesn't fit when we perform new experiment where we try to 'tweek' one of the parameters). Then in, what way was the model an explaination? 

Should everything be expressed as an ODE model? As we add more components to decribe anomalies in out data, the model becomes more and more complicated and it becomes easier and easier to fix multiple different parameters to the same data. Hence, we are in a balance between explainability and predictability. The more complex our model is, the less explaination it gives and the more

If we develop a black box model that accurately predicts the data, isn't that pretty good? Moreover, there *are* ways to tease how what the model has learned. This is where bayesian portion comes in, we can determine where exactly the uncertainty is for the outputs it generates, which is immensely useful.

In [2]:
from caldera.data import GraphData
from typing  import *


class Transition(object):
    
    __slots__ = ['state', 'action', 'next_state', 'reward']
    
    def __init__(self, state, action, next_state, reward):
        self.state = state
        self.action =  action
        self.next_state = next_state
        self.rewarrd = reward
        
        
class GraphTransition(Transition):
    pass


class ReplayMemory(object):

    def __init__(self, capacity, transition_type: Type[Transition] = Transition):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        self.transition_type = transition_type
 

    def push(self, *args):
            """Saves a transition."""
            if len(self.memory) < self.capacity:
                self.memory.append(None)
            self.memory[self.position] = self.transition_type(*args)
            self.position = (self.position + 1) % self.capacity

        def sample(self, batch_size):
            return random.sample(self.memory, batch_size)

        def __len__(self):
            return len(self.memory)
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 36)

In [3]:
class Environment(object):
    """The environment the designer Agent exists in"""
    
    def __init__(self):
        self._memory = None
        self.model = None
        self.state = None
        
    def set_state(self, state):
        self.state = state
    
    def attach_memory(self, memory):
        self._memory = memory
    
    def step(self, action: int):
        """Select and perform the given action"""
        # this is where we modify the circuit, re-run the simulation
        # if memory is attached, we save the Transition
        # we return the Transition and reward
        
        # Progression:
        # step 1: we start off with a simple Oracle simulation that is 100% accurate
        # step 2: we use learned uncertain parameters from results, there is a penalty for usage of 
        # uncertain parts, but this is a parameter we can explore.
        pass

In [ ]:
from caldera.models import GraphEncoder, GraphCore

In [9]:
import torch
from torch import nn
from caldera.blocks import MLP

class DQN(nn.Module):
    """
    We will be given a GraphBatch instance which will contain an
    arbitrary number of graphs.
    """
    def __init__(self, n_actions):
        super().__init__()
        self.encoder = torch.nn.Sequential(MLP(16, n_actions))

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, graph_data, desired_behavior):
        """Here we take in the current graph and desired_behavior and return the probability
        of actions."""
        return self.encoder(graph_data)


In [12]:
BATCH_SIZE = 10
n_actions = 10
policy_net = DQN(n_actions)
target_net = DQN(n_actions)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


def select_action(state):
    # this is a comment 
    # this is a comment 
    # this is another comments


SyntaxError: unexpected EOF while parsing (<ipython-input-12-97adb90d885b>, line 17)

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return 
    transitions = memory.sample(BATCH_SIZE)
    
    batch = Transition(*zip(*transitions))

**Actions:**


* Swap node 
* Add node
* swap nodes
* swap edges
* Remove node
* Add edge``
* Terminate




## action space

here we define the action space as modifications to the graph representing the circuit network



In [3]:
def select_action():
    """selects the next action """

    passlamar;lamar


def select_action():
    pass






